In [1]:
import requests
from urllib.parse import urlencode
import pandas as pd

# repalce MY_API by yours
# api can be obtained from newsAPI website
MY_API = '***********************'

In [2]:
def get_article_info(article):
    # extract information from articles
    article_keys = ['source', 'author', 'title', 'description', 'url', 'urlToImage', 'publishedAt', 'content']
    result = []
    for key in article_keys:
        # source is a dict, not a string
        if key == 'source':
            result.append(article[key]['id'])
            result.append(article[key]['name'])
        else:
            result.append(article[key])
    return result

In [4]:
# information of S&Q 500 companies
sq500 = pd.read_csv('../data/sq500.csv')

In [6]:
sq500.head(5)

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Biotechnology,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989


In [7]:
colname = ['source_id', 'source_name', 'author', 'title', 'description', 'url', 'urlToImage', 'publishedAt', 'content', 'Security']
base_url = 'https://newsapi.org/v2/everything'
sq500_articles = []
for idx, row in sq500.iterrows():
    # request to newAPI is limited by 100 per day
    # we only send 3 requests for testing
    if idx > 3:
        break
    # security is the name of the company
    security = row['Security']
    params = {
    'q': security,
    'from': '2024-01-12',
    'sortBy': 'relevancy',
    'language': 'en',
    'apiKey': MY_API}
    # encode the parameters so it fit the url format
    query_string = urlencode(params)
    url = f"{base_url}?{query_string}"
    response = requests.get(url)
    json = response.json()
    
    # for each company, we should get a few articles
    # we extract the results to a list
    for article in json['articles']:
        result = get_article_info(article)
        result.append(security)
        sq500_articles.append(result)
df = pd.DataFrame(sq500_articles, columns=colname)
df.to_csv('../data/sq500_articles_test.csv', index=False)

KeyError: 'articles'